<a href="https://colab.research.google.com/github/MuhammadBasilKhan/Fine-tune-model/blob/main/fracture_vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D,Flatten,MaxPooling2D,Dense,Dropout
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential

In [ ]:
dir='/content/drive/MyDrive/drive/fracture data model/images'


In [ ]:
img_size = (224, 224)

# Set the batch size for training
batch_size = 32

# Create data generators for training and validation
train_data_gen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
x = []
y = []
for label, folder_name in enumerate(['Fractured', 'Non_fractured']):
    folder_path = os.path.join(dir, folder_name)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, img_size)
            x.append(img)
            y.append(label)


In [ ]:
x = np.array(x)
y = np.array(y)

In [ ]:
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.2, stratify=y)

In [ ]:
x_train.shape

(3266, 224, 224, 3)

In [ ]:
x_val.shape

(817, 224, 224, 3)

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
vgg=VGG16()

553467096/553467096 [==============================] - 7s 0us/step


In [ ]:
from keras import Sequential
model=Sequential()

In [ ]:
for layer in vgg.layers[:-1]:
  model.add(layer)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [ ]:
for layer in model.layers:
  layer.trainable=False

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [ ]:
from keras.layers import Dense

In [ ]:
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=5,validation_data=(x_val,y_val))

Epoch 1/5
103/103 [==============================] - 2655s 26s/step - loss: 0.4113 - accuracy: 0.8374 - val_loss: 0.3251 - val_accuracy: 0.8654
Epoch 2/5
103/103 [==============================] - 2638s 26s/step - loss: 0.3308 - accuracy: 0.8686 - val_loss: 0.3298 - val_accuracy: 0.8605
Epoch 3/5
103/103 [==============================] - 2620s 25s/step - loss: 0.2960 - accuracy: 0.8852 - val_loss: 0.3192 - val_accuracy: 0.8531
Epoch 4/5
103/103 [==============================] - 2593s 25s/step - loss: 0.2742 - accuracy: 0.8944 - val_loss: 0.3871 - val_accuracy: 0.8384
Epoch 5/5
103/103 [==============================] - 2645s 26s/step - loss: 0.2625 - accuracy: 0.8990 - val_loss: 0.5383 - val_accuracy: 0.7479


In [ ]:
model.evaluate(x_val,y_val)

26/26 [==============================] - 540s 21s/step - loss: 0.5383 - accuracy: 0.7479


[0.5383432507514954, 0.7478579878807068]

In [ ]:
model.save('fracturevgg16.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
